In [32]:
import csv
import numpy as np

from glob import glob
import shutil
import hashlib

def get_early_hash(fn):
    BUF_SIZE = 65536*128
    h = hashlib.new("sha256")
    with open(fn, "rb") as f:
        data = f.read(BUF_SIZE)
        h.update(data)

    return h.hexdigest()

def replace(ss, newcode):
    l = ss.split("/")
    ll = l[-1].split("-")
    return ss.replace(f"-{ll[2]}-", f"-{newcode}-")

In [33]:
# 변환 기록 버전
dir_list = ["DRWS", "DSRT", "NORM", "PHON", "WPHN"]

nn = 2

with open(f"map_{nn:03d}.txt", "w") as f:
    for dd in dir_list:
        flist1 = glob(f"./DRIVER{nn:03d}-done/{dd}/*.*")
        flist1.sort()
        #flist2 = glob(f"./DRIVER{nn:03d}-done/{dd}/*.obd")

        for ff in flist1:
            fnew = replace(ff, dd)
            f.write(f"{ff},{fnew}\n")
            shutil.move(ff,fnew)
            
        #for ff in flist2:
        #    fnew = replace(ff, dd)
        #    f.write(f"{ff},{fnew}\n")
        #    shutil.move(ff,fnew)

임시. 기존 파일의 체크섬 계산

In [34]:
nn = 2
with open(f"hashORG{nn:03d}.txt", "w") as f:
    dd = f"driver{nn:03d}"
    flist1 = glob(f"./ORG/{dd}/rec1/*.*")

    for ff in flist1:
        f.write(f"{ff},{get_early_hash(ff)}\n")

파일 이름 변환

원본을 덮어쓰니 주의

In [35]:
dir_list = ["DRWS", "DSRT", "NORM", "PHON", "WPHN"]

nn = 2

with open(f"hash{nn:03d}.txt", "w") as f:
    for dd in dir_list:
        flist1 = glob(f"./DRIVER{nn:03d}-done/{dd}/*.mp4")
        flist2 = glob(f"./DRIVER{nn:03d}-done/{dd}/*.obd")

        for ff in flist1:
            f.write(f"{ff},{get_early_hash(ff)}\n")
            #shutil.move(ff, replace(ff, dd))
            
        for ff in flist2:
            f.write(f"{ff},{get_early_hash(ff)}\n")
            #shutil.move(ff, replace(ff, dd))

파일 변환 매치

In [36]:
for nn in [2]:
    arr1org = np.array(list(csv.reader(open(f"hashORG{nn:03d}.txt", "r"))))
    arr1done = np.array(list(csv.reader(open(f"hash{nn:03d}.txt", "r"))))

    arr_sorted, i1o, i1d = np.intersect1d(arr1org[:,1], arr1done[:,1], assume_unique=False, return_indices=True)

    with open(f"map{nn:03d}.txt", "w") as f:
        f.write("ORG -> NEW\n")
        for (i1, i2) in zip(i1o, i1d):
            f.write(f"{arr1org[i1,0]},{arr1done[i2,0]}\n")

In [37]:
len(arr1org)

3204

In [38]:
len(arr1done)

3076

In [39]:
len(i1o)

2675

In [40]:
len(i1d)

2675

In [ ]:
arr_sorted, i1o, i1d = np.intersect1d(arr1org[:,0], arr1done[:,1], assume_unique=False, return_indices=True)